In [2]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

%load_ext autoreload

transform = transforms.Compose([
    transforms.ToTensor(),  # convert the data to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # normalize the data
])

train_data = datasets.MNIST(root='./data/.cache', train=True, transform=transform, download=True)
test_data = datasets.MNIST(root='./data/.cache', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:02<00:00, 4134580.79it/s]


Extracting ./data/.cache/MNIST/raw/train-images-idx3-ubyte.gz to ./data/.cache/MNIST/raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 5778547.62it/s]


Extracting ./data/.cache/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/.cache/MNIST/raw



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 4707618.28it/s]


Extracting ./data/.cache/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/.cache/MNIST/raw



100%|██████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 782426.84it/s]

Extracting ./data/.cache/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/.cache/MNIST/raw



In [3]:
import matplotlib.pyplot as plt

data, targets = next(iter(train_loader))

x_train = data.T.squeeze().numpy().reshape(784, 32)
y_train = targets.T.numpy().reshape(1, 32)

print(f'label: {targets[0].item()}')
plt.imshow(data[0].squeeze(), cmap='gray');


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [326]:
import numpy as np
import simple_nn as nn
import util
import torch.nn.functional as F

%autoreload 2

model = nn.Sequential([
    nn.LinearLayer(784, 50),
    nn.LeakyRelu(),
    nn.LinearLayer(50, 10),
    # TODO(d): figure out how to deal with softmax and cross-entropy
#     nn.Softmax(axis=0),
])

epochs = 100
lr = 1e-3

def get_accuracy(preds, y):
    return np.sum(preds == y) / y.size

targets = np.zeros((y_train.max()+1, y_train.size))
targets[y_train, np.arange(y_train.shape[1])] = 1

for n in range(epochs):
    logits = model.forward(x_train)
    probs = util.softmax(logits, axis=0)
    loss = -1/len(probs) * np.sum(targets * np.log(probs))
    grad = probs - targets
    if n % 10 == 0:
        preds = np.argmax(logits, 0)
        acc = get_accuracy(preds, targets)
        print(f'{n}: {loss}')
    model.backward(grad)
    model.update_params(lr)

    
def predict(image):
    logits = model.forward(image)
    return np.argmax(logits)

print(x_train[:,[0]].shape)
print(predict(x_train[:,[2]]), y_train[:,2].item())


0: 680.3021227197935
10: 8.608882117623208
20: 0.0008460515595192267
30: 0.0004124621650431659
40: 0.0002827946670317765
50: 0.00021818961198482616
60: 0.00017888739722447984
70: 0.00015222975714455656
80: 0.00013285903203872205
90: 0.00011809582645017547
(784, 1)
4 4
